# Jigsaw Unintended Bias in Toxicity Classification

1. Import dependencies

In [1]:
import random
import time

import numpy as np
import torch
import os

from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

from src.model.metrics import compute_metrics
from src.data.toxicity import load_toxicity_dataset
from src.util.torch_device import resolve_torch_device
from src.definitions import MODELS_FOLDER, EXTERNAL_DATA_FOLDER, PROCESSED_DATA_FOLDER

/Users/alexandermelashchenko/Workspace/nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


2. Prepare Env

In [2]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

model_checkpoint = "distilbert/distilbert-base-cased"

epoch_time = int(time.time())

os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

3. Load dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

ds = load_toxicity_dataset(
    EXTERNAL_DATA_FOLDER, PROCESSED_DATA_FOLDER, tokenizer, random_seed
)

Map:   0%|          | 0/1624383 [00:00<?, ? examples/s]

Map:   0%|          | 0/180488 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1624383 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/180488 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/1624383 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/180488 [00:00<?, ? examples/s]

In [4]:
# ds["train"]["label"]

In [5]:
# from datasets import load_dataset

# imdb_data = load_dataset("imdb")


# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_imdb = imdb_data.map(tokenize_function, batched=True)

# tokenized_imdb["train"]["label"]

In [6]:
# tokenized_imdb["train"]["label"]

In [7]:
label2id = {
    "non-toxic": 0,
    "toxic": 1,
}

id2label = {v: k for k, v in label2id.items()}

3. Prepare model

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
).to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir=MODELS_FOLDER / "span-detection-checkpoint",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=random_seed,
    auto_find_batch_size=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1624383
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 180488
    })
})


In [10]:
print(ds["train"]["label"][0])

0


4. Train model

In [ ]:
trainer.train()

5. Evaluate

In [ ]:
evaluation_feedback = trainer.evaluate()

6. Save weights

In [ ]:
trainer.save_model(MODELS_FOLDER / "jigsaw-unintended-bias-in-toxicity-classification")
tokenizer.save_pretrained(
    MODELS_FOLDER / "jigsaw-unintended-bias-in-toxicity-classification"
)